In [1]:
""" 
IMPORTS
"""
import os
import autograd.numpy as np
import pickle

from one.api import ONE
from pprint import pprint
import matplotlib.pyplot as plt
import concurrent.futures

# Get my functions
functions_path =  '/home/ines/repositories/representation_learning_variability/Models/Sub-trial//2_fit_models/'
#functions_path = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability//Models/Sub-trial//2_fit_models/'
os.chdir(functions_path)
from preprocessing_functions import idxs_from_files_pre_custom_qc, idxs_from_files, prepro_design_matrix, concatenate_sessions
functions_path =  '/home/ines/repositories/representation_learning_variability/Functions'
#functions_path = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability//Models/Sub-trial//2_fit_models/'
os.chdir(functions_path)
from data_processing import save_and_log

one = ONE(mode="remote")

## Parameters

In [2]:
bin_size = 0.1
# bin_size = 0.02
bin_size = np.round(1/60, 3)  # No binning, number indicates sampling rate

# Plotting params
multiplier = 1/bin_size
multiplier = 1

event_type_list = ['goCueTrigger_times']  # , 'feedback_times', 'firstMovement_times'
event_type_name = ['Go cue']  # , 'Feedback time', 'First movement onset'


In [3]:
data_file = "preprocessed_data_v5_01-17-2025"

if data_file == "preprocessed_data_v4_170724":
    use_sets = [['avg_wheel_vel'], ['Lick count'], ['whisker_me'],
                ['left_X', 'left_Y', 'right_X', 'right_Y'], ['nose_X', 'nose_Y']]
    var_interest_map = ['avg_wheel_vel', 'Lick count', 'whisker_me', 'left_X', 'nose_X']
    idx_init_list = [0, 1, 2, 3, 7]
    idx_end_list = [1, 2, 3, 7, 9]
    
    # Load preprocessed data
    prepro_results_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Results/90_trials/' + str(bin_size) + '/'
    idxs, mouse_names, matrix_all, matrix_all_unnorm, session_all = pickle.load(open(prepro_results_path + data_file, "rb"))
    collapsed_matrices, collapsed_unnorm, collapsed_trials = concatenate_sessions (mouse_names, matrix_all, matrix_all_unnorm, session_all)
    
elif data_file == "preprocessed_data_v4_171224_alltrials":
    use_sets = [['avg_wheel_vel'], ['whisker_me'], ['Lick count']]
    var_interest_map = ['avg_wheel_vel', 'whisker_me', 'Lick count']
    idx_init_list = [0, 1, 2]
    idx_end_list = [1, 2, 3]
    
    # Load preprocessed data
    prepro_results_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Results/' + str(bin_size) + '/'
    idxs, mouse_names, matrix_all, matrix_all_unnorm, session_all = pickle.load(open(prepro_results_path + data_file, "rb"))
    collapsed_matrices, collapsed_unnorm, collapsed_trials = concatenate_sessions (mouse_names, matrix_all, matrix_all_unnorm, session_all)

elif data_file == "preprocessed_data_v5_01-17-2025":
    use_sets = [['avg_wheel_vel'], ['whisker_me'], ['Lick count'], ['0.25', '0.5',
       '1.0', '2.0', '4.0', '8.0', '16.0']]
    var_interest_map = ['avg_wheel_vel', 'whisker_me', 'Lick count', '0.25', '0.5',
       '1.0', '2.0', '4.0', '8.0', '16.0']
    idx_init_list = [0, 1, 2, 3]
    idx_end_list = [1, 2, 3, 9]
    
    # Load preprocessed data
    prepro_results_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Results/' + str(bin_size) + '/'
    filename = prepro_results_path + data_file
    idxs, mouse_names, matrix_all, matrix_all_unnorm, session_all = pickle.load(open(filename, "rb"))
    collapsed_matrices, collapsed_unnorm, collapsed_trials = concatenate_sessions (mouse_names, matrix_all, matrix_all_unnorm, session_all)

    

In [9]:
# Load preprocessed data
results_path = '/home/ines/repositories/representation_learning_variability/Video and wheel/Video QC/'
filename = str(results_path + 'custom_qc01-07-2025')
custom_qc = pickle.load(open(filename, "rb"))

,task,dlcBody,dlcLeft,behavior,dlcRight,videoBody,videoLeft,videoRight,_task_iti_delays,_videoBody_focus,...,eid,_experimenter_task,_videoLeft_video_present,_videoRight_video_present,videoRight`,_videoRight`_pin_state,_videoRight`_wheel_alignment,experimenter,ratio,miss_lick_count
0,WARNING,CRITICAL,FAIL,1,FAIL,CRITICAL,WARNING,FAIL,0.582547,True,...,56956777-dca5-468c-87cb-78150432cc57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.32,15.0
0,PASS,PASS,FAIL,1,FAIL,WARNING,WARNING,WARNING,0.583333,True,...,6ed57216-498d-48a6-b48b-a243a34710ea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.69,66.0
0,PASS,PASS,FAIL,1,FAIL,WARNING,WARNING,WARNING,0.147351,True,...,fa1f26a1-eb49-4b24-917e-19f02a18ac61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.24,7.0
0,WARNING,PASS,FAIL,1,PASS,WARNING,WARNING,WARNING,0.179487,True,...,91a3353a-2da1-420d-8c7c-fad2fedfdd18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.56,51.0
0,WARNING,PASS,WARNING,1,FAIL,WARNING,WARNING,WARNING,0.482679,True,...,5157810e-0fff-4bcf-b19d-32d4e39c7dfc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.29,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,WARNING,CRITICAL,FAIL,1,PASS,CRITICAL,FAIL,WARNING,0.214829,True,...,c557324b-b95d-414c-888f-6ee1329a2329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.33,33.0
0,PASS,PASS,FAIL,1,FAIL,WARNING,FAIL,WARNING,0.519856,True,...,5d01d14e-aced-4465-8f8e-9a1c674f62ec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.38,55.0
0,PASS,PASS,FAIL,1,FAIL,WARNING,FAIL,WARNING,0.199637,True,...,81a78eac-9d36-4f90-a73a-7eb3ad7f770b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.3,76.0
0,PASS,PASS,FAIL,1,FAIL,WARNING,FAIL,WARNING,0.296601,True,...,626126d5-eecf-4e9b-900e-ec29a17ece07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.11,50.0


In [17]:
data_path_v1 =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/' + 'v1/' + str(bin_size) + '/'
data_path_v2 =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/' + 'v2_3Nov2023/' + str(bin_size) + '/'
#data_path_v2 =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/' + 'v2_3Nov2023/' + str(bin_size) + '/'
data_path_v3 =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/' + 'v3_1Jul2024/' + str(bin_size) + '/'
data_path_v4 =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/' + 'v4_5Jul2024/' + str(bin_size) + '/'
# data_path_v4 =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/' + 'v4_21oct2024/' + str(bin_size) + '/'
data_path_v4 =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/' + 'v4_9Dec2024/' + str(bin_size) + '/'
data_path_v5 =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/' + 'v5_15Jan2025/' + str(bin_size) + '/'

data_path = data_path_v5

os.chdir(data_path)
design_matrices = os.listdir(data_path)

if data_path == data_path_v1:
       var_names = ['avg_wheel_vel', 'pupil_diameter',
              'whisker_me', 'nose_speed_X', 'nose_speed_Y', 
              'l_paw_speed_X', 'l_paw_speed_Y', 'pupil_speed_X',
              'pupil_speed_Y', 'Gaussian_licks']
elif data_path == data_path_v2:
       var_names = ['avg_wheel_vel', 'pupil_diameter',
              'whisker_me', 'nose_speed', 'l_paw_speed', 'left_X', 'left_Y',
              'pupil_speed', 'pupil_X', 'pupil_Y', 'Lick count']
       # var_names = ['avg_wheel_vel', 'whisker_me']
       var_names = ['avg_wheel_vel']
elif data_path == data_path_v3:
       var_names = [ 'wheel_final', 'Lick count', 'pupil_diameter', 'whisker_me',
       'left_X', 'left_Y', 'pupil_X', 'pupil_Y', 'nose_X', 'nose_Y']
       var_names = ['Lick count']
elif data_path == data_path_v4:
       var_names = ['nose_X', 'nose_Y']
       # var_names = ['right_X', 'right_Y']
       var_names = ['avg_wheel_vel']
       var_names = ['avg_wheel_vel', 'whisker_me', 'Lick count']
elif data_path == data_path_v5:
       var_names = ['avg_wheel_vel', 'whisker_me', 'Lick count', '0.25', '0.5',
       '1.0', '2.0', '4.0', '8.0', '16.0']




In [12]:

pre_custom_qc = False
first_90 = False

if pre_custom_qc:
       # Load frame rate
       fr_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/frame_rate/'
       os.chdir(fr_path)
       frame_rate = pickle.load(open(fr_path + "frame_rate", "rb"))
       
       # use_vars = ['avg_wheel_vel', 'Lick count', 'whisker_me',
       #        'left_X', 'left_Y', 'right_X', 'right_Y', 'nose_X', 'nose_Y']
       idxs, mouse_names = idxs_from_files_pre_custom_qc(one, design_matrices, frame_rate, data_path, bin_size)
       matrix_all, matrix_all_unnorm, session_all = prepro_design_matrix(one, idxs, mouse_names, 
                                                                      bin_size, multiplier, var_names, data_path, first_90)
else:
       idxs, mouse_names = idxs_from_files(design_matrices, bin_size)
       matrix_all, matrix_all_unnorm, session_all = prepro_design_matrix(one, idxs, mouse_names, 
                                                                      bin_size, multiplier, var_names, data_path, first_90)


# Save
to_save = idxs, mouse_names, matrix_all, matrix_all_unnorm, session_all


file_to_save = to_save
filename = "preprocessed_data_v5_"
save_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Results/' + str(bin_size) + '/'
file_format = 'pickle'
script_name = 'B_preprocess_data.ipynb'
metadata = save_and_log(file_to_save, filename, file_format, save_path, script_name)


 

local md5 mismatch on dataset: churchlandlab/Subjects/CSHL049/2020-01-11/001/alf/_ibl_trials.stimOff_times.npy


Downloading: /home/ines/Downloads/ONE/alyx.internationalbrainlab.org/churchlandlab/Subjects/CSHL049/2020-01-11/001/alf/_ibl_trials.stimOff_times.c07b94b4-18c6-4940-9bd6-9661f4ae5ceb.npy Bytes: 4600


100%|██████████| 0.00438690185546875/0.00438690185546875 [00:00<00:00,  2.42it/s]


AttributeError: 'tuple' object has no attribute 'copy'

In [1]:
mouse_names

NameError: name 'mouse_names' is not defined

In [ ]:
# # Save most_likely_states
# prepro_results_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Results/' + str(bin_size) + '/'
# os.chdir(prepro_results_path)
# pickle.dump(to_save, open("preprocessed_data_v4_171224_alltrials", "wb"))  